# 12. 虛擬環境與套件 (Virtual Environments and Packages)

* 身為一個軟體工程師，我們每天都在處理「環境」問題，而虛擬環境（Virtual Environments）可以說是在 Python 領域中，區分「業餘愛好者」與「專業開發者」的第一道分水嶺。

## 12.1. 簡介：為什麼我們「必須」使用虛擬環境？

---
* 官方文件提到了應用程式 A 和 B 的版本衝突。這在實務上被稱為「**依賴地獄 (Dependency Hell)**」，這是每個工程師的惡夢。

* 實務上的情境（The "Why"）

    * 想像一下你的電腦（或公司的伺服器）是一間共用的廚房（這就是你的「全域 Global」Python 環境）。

        1. 專案 A：舊的維護專案 (Legacy Project)

            * 你正在維護一個三年前上線的 Django 網站。

            * 它當初是基於 Django 2.2 和 requests 2.18.0 開發的。它能穩定運作，但你不敢輕易升級它，因為一升級，天知道會壞掉多少功能。

        2. 專案 B：新的 AI 專案 (New Project)

            * 你同時在開發一個新的 AI 應用（呼應你的興趣），你需要使用最新的 TensorFlow 2.10。

            * TensorFlow 2.10 在安裝時，它「依賴」了 requests 2.25.0 或更新的版本。

* 衝突發生了：

    * 如果你在「共用廚房」中，為了專案 B，把 requests 從 2.18.0 升級到 2.25.0，那麼專案 A（舊網站）可能就立刻崩潰了。因為新版的 requests 可能移除了某個舊專案正在使用的函式。

* 解決方案：虛擬環境（The "Solution"）

    * 虛擬環境，就是為每個專案打造一個「獨立的私廚」。

        * `venv` 所做的，就是幫你在專案資料夾裡建立一個「私廚」（例如 `.venv` 資料夾）。

        * 這個私廚裡有專屬於它自己的廚具（一個獨立的 Python 直譯器複本）和食材櫃（`site-packages` 資料夾）。

    * 當你「啟動 (activate)」這個環境時，你就走進了這個私廚。

        * 你在專案 A 的私廚裡，安裝的是 Django 2.2 和 requests 2.18.0。

        * 你離開 (deactivate)，走進專案 B 的私廚，安裝的是 TensorFlow 2.10 和 requests 2.25.0。

* 兩者完全隔離 (Isolated)，互不干擾。

* **專業開發者的核心價值**：

1. 隔離性 (Isolation)： 這是最直接的好處。你的專案不會「污染」你的全域 Python，反之亦然。

2. 可重現性 (Reproducibility)： 這是更重要的一點。當你的同事（或未來的你）要接手這個專案時，你如何確保他們能 100% 複製你的開發環境？

    * 你會使用 `pip freeze > requirements.txt` 指令，把你「私廚食材櫃」裡的所有套件與其精確版本，列成一張清單。

    * 你的同事拿到專案後，只需建立他們自己的私廚（`venv`），然後用 `pip install -r requirements.txt` 照單採購，就能完美重現你的環境。

* 重點總結： 虛擬環境的核心價值是「**隔離**」與「**可重現性**」。
    * 絕對不要在你的系統（全域）Python 上用 `pip install` 來開發專案。


## 12.2. 建立虛擬環境：實務上的最佳實踐 (Best Practices)

---
官方教學

* 用來建立與管理虛擬環境的模組叫做 `venv`。

    * `venv` 通常會安裝你能夠取得的最新版本的 Python。
    
    * 要是你的系統有不同版本的 Python，你可以透過 python3 這個指令選擇特定或是任意版本的 Python。

* 在建立虛擬環境的時候，在你決定要放該虛擬環境的資料夾之後，以腳本 (script) 執行 `venv` 模組並且給定資料夾路徑：

```
python -m venv tutorial-env
```

* 如果 `tutorial-env` 不存在的話，這會建立 `tutorial-env` 資料夾，並且也會在裡面建立一個有 Python 直譯器的複本以及不同的支援檔案的資料夾。

* 虛擬環境的常用資料夾位置是 `.venv`。這個名稱通常會使該資料夾在你的 shell 中保持隱藏，因此這樣命名既可以解釋資料夾存在的原因，也不會造成任何困擾。它還能防止與某些工具所支援的 `.env` 環境變數定義檔案發生衝突。

* 一旦你建立了一個虛擬環境，你可以啟動它。

    * 在 Unix 或 MacOS 系統，使用：

    ```
    source tutorial-env/bin/activate
    ```

    * （這段程式碼適用於 bash shell。如果你是用 csh 或者 fish shell，應當使用替代的 activate.csh 與 activate.fish 腳本。）



---
官方文件教了你「如何做」，我來補充「業界習慣怎麼做」和「為什麼這樣做」。

1. 該用 `venv` 還是 `virtualenv`？

    * 你可能在舊的教學或專案中看過 `virtualenv`。這是一個第三方的套件。

    * `venv` 是從 Python 3.3 開始內建在標準函式庫中的模組。

    * 實務建議： 除非你必須支援 Python 2（現在幾乎不需要了），永遠優先使用 `venv`。它內建、輕量，而且是官方標準。

2. 指令的細微差異

    * 官方指令是：`python -m venv tutorial-env`

    * `python -m venv`：為什麼用 `-m`？

        * `-m` 的意思是「module」。這是在告訴 Python：「請你幫我以模組的形式執行 `venv`」。

        * 這比直接執行某個 `venv.py` 腳本更穩健。它能確保你使用的是你當前 python 指令所對應的那個 `venv` 模組。

        * 實務情境： 如果你的系統上同時裝了 python3.9 和 python3.11。

            * 執行 `python3.9 -m venv .venv` 會建立一個基於 Python 3.9 的環境。

            * 執行 `python3.11 -m venv .venv` 則會建立一個基於 Python 3.11 的環境。

        * 這讓你對要建立「哪個版本」的 Python 環境有精確的控制。

3. 虛擬環境該放哪裡？命名慣例

    * 官方用了 `tutorial-env` 這個名字，但這在實務上非常少見。

    * 業界標準： 將虛擬環境資料夾建立在專案的根目錄下，並將其命名為 `.venv`。

    ```
    my_awesome_project/
    ├── .venv/            <-- 你的虛擬環境（私廚）
    ├── .gitignore
    ├── requirements.txt  <-- 你的套件採購清單
    ├── src/
    │   └── main.py
    └── README.md
    ```

    * 為什麼用 `.venv`？

        1. 隱藏性： 在 Unix/Linux/macOS 中，以 `.` 開頭的檔案/資料夾預設是隱藏的。這很好，因為虛擬環境是「支援性」的工具，不是你專案的「原始碼」，你平常不需要在檔案總管裡看到它。

        2. 標準化： 越來越多的工具（例如 VS Code 的 Python 擴充套件）會自動偵測名為 `.venv` 的資料夾。當你用 VS Code 開啟 my_awesome_project 時，它會自動發現 `.venv` 並詢問你是否要切換到這個環境的 Python 直譯器。

4. 絕對、絕對要做的：`.gitignore`

    * `venv` 裡面發生了什麼事？它複製了 Python 直譯器，並且在 site-packages 裡安裝了 數百 MB 甚至 數 GB 的套件（特別是 AI 相關的）。

    * 這些是「可重現」的，你絕對不該把 `.venv` 資料夾 commit 到 Git 裡面。

    * 所以，在你建立 `.venv` 的同時，你必須立刻去編輯專案根目錄的 `.gitignore` 檔案，加上這一行：

    ```
    # .gitignore

    # Python Virtual Environment
    .venv/
    ```

    * 這樣 Git 就會永遠忽略這個資料夾。你的同事會感謝你，因為他們不需要下載你那 2GB 的 tensorflow 套件，他們只需要拿著你的 `requirements.txt` 清單自己去 pip install 就好。

5. 啟動 (Activate) 與提示字元

* 官方文件的啟動指令是正確的：

    * Unix/macOS: `source .venv/bin/activate`

    * Windows (PowerShell): `.venv\Scripts\Activate.ps1`

        * 實務陷阱： 在 PowerShell 中，你可能需要先執行 `Set-ExecutionPolicy -ExecutionPolicy RemoteSigned -Scope Process` 來允許執行腳本。

    * Windows (cmd.exe): `.venv\Scripts\activate.bat`

* 啟動到底做了什麼？

    * 當你看到 Shell 提示字元前面多了 (`.venv`)，這不只是個提示。

    * `activate` 腳本的核心動作是：**暫時修改你 Shell 的 `PATH` 環境變數。**

        * 啟動前： 你的 `PATH` 可能長這樣：`/usr/bin:/usr/local/bin:...`

            * 當你輸入 `python`，系統會去 `/usr/bin` 找到全域的 Python。

        * 啟動後： 腳本會把你的 `.venv` 路徑加到 `PATH` 的最前面：`</path/to/project/.venv/bin>:/usr/bin:/usr/local/bin:...`

            * 現在當你輸入 `python` 或 `pip`，系統會優先在 `</path/to/project/.venv/bin>` 裡尋找，於是就找到了你「私廚」裡的那個 Python 直譯器。

* 這就是「隔離」的魔法所在。而 `deactivate` 指令（退出環境）所做的，就是把 `PATH` 改回去而已。

---
總結你的「專業工作流」 (Workflow)

1. 建立新專案： `mkdir my_project && cd my_project`

2. 建立環境： `python3 -m venv .venv` (使用標準名稱 `.venv`)

3. 忽略環境： `echo ".venv/" >> .gitignore` (立刻告訴 Git 忽略它)

4. 啟動環境： `source .venv/bin/activate`

5. 安裝套件： `pip install requests pandas` (這時會裝在 `.venv` 裡)

6. 撰寫程式： `code .` (打開 VS Code，它會自動找到 `.venv`)

7. 產生清單： `pip freeze > requirements.txt` (準備交給同事)

8. 版本控制： `git add .`、`git commit` (提交你的程式碼和 `requirements.txt`，但不是 `.venv`)

9. 離開環境： `deactivate` (回到全域環境)

## 12.3. 用 pip 管理套件 (Managing Packages with pip)

* 如果你在上一個階段學會的 `venv`（虛擬環境）是為你的專案打造的「獨立私廚」，那麼 `pip` 就是你專屬的「食材採購總管」。

* `pip` 負責與「**PyPI (Python Package Index)**」這個全球最大的「Python 食材批發市場」打交道，幫你精確地取得、管理、更新你烹飪（開發）所需的各種「食材」（也就是套件，Packages）。

* 讓我們來實務解析 `pip` 的核心指令。

---
1. `pip install`: 採購食材

這是你最常用的指令。它有幾種不同的「採購模式」。

模式一：`pip install novas` (採購最新品)

* 官方說明： 安裝最新版本的套件。

* 實務情境 (「總管，幫我買最新的『novas』」)：

    * 你正在開發一個新功能，需要一個你從未用過的工具（例如，一個用於處理日期的 `arrow` 套件）。

    * 你執行 `pip install arrow`。`pip` 會去 PyPI 市場上找到 `arrow`，並且直接拿最新版本的貨回來，放進你的 `.venv/lib/site-packages`（私廚食材櫃）裡。

    * 重點： 你通常只在「首次」為專案添加新依賴時這樣做。

模式二：`pip install requests==2.6.0` (按訂單採購)

* 官方說明： 安裝特定版本的套件。

* 實務情境 (「總管，我要的『requests』必須是 2.6.0 版，一模一樣」)：

    * 在實務上，你極少會手動輸入這個指令。

    * 這個指令的真正用途是給「自動化腳本」或「`requirements.txt`」使用的（後面會詳述）。

    * 唯一的例外是：你安裝了最新版的 `requests` (例如 2.7.0)，結果它把你的舊功能搞壞了（這在軟體工程中稱為「Regression (功能退回)」）。你知道 2.6.0 版是正常的，所以你手動執行 `pip install requests==2.6.0` 來降級 (Downgrade)，以鎖定在一個已知的穩定版本。

模式三：`pip install --upgrade requests` (食材升級)

* 官方說明： 把套件升級到最新的版本。

* 實務情境 (「總管，去看看市場上的『`requests`』有沒有新貨，有的話幫我換掉」)：

    * 警告： 這在一個穩定的專案中可能是個危險操作。

    * 你不會隨便在你的「生產環境 (Production)」或「主分支 (main branch)」上執行這個指令。

    * 正確的工作流程是：

        1. 在一個新的 Git 分支上 (例如 `feature/upgrade-requests`)。

        2. 啟動你的虛擬環境。

        3. 執行 `pip install --upgrade requests`。

        4. 執行你的完整測試 (例如 `pytest`)，確保所有功能在新版套件下依然能正常運作。

        5. 如果測試通過，你才會更新 `requirements.txt` 並合併 (merge) 你的程式碼。

---
2. `pip uninstall`: 清理食材櫃

* 官方說明： 移除套件。

* 實務情境 (「總管，這個『novas』我再也用不到了，幫我丟掉」)：

    * 你當初為了某個實驗性功能安裝了 `novas`，但最後你決定用別的方法實現，這個套件就成了「無用依賴 (Unused Dependency)」。

    * 保持環境清潔是好習慣。執行 `pip uninstall novas`，`pip` 會問你「`Proceed (y/n)?`」，確認後就會幫你刪除它。

* 重點： 刪除後，記得要去更新你的 `requirements.txt` 清單。

---
3. `pip show`, `list`, `freeze`: 盤點庫存

這三個指令都是在「盤點」，但目的截然不同，實務上很容易搞混。

`pip list` (給「人」看的快速清單)

* 官方說明： 顯示所有已安裝的套件。

* 實務情境 (「總管，隨便給我看一下食材櫃裡有啥」)：

    * 你只是想快速瞄一眼你「當前環境」裝了哪些東西。

    * 它的輸出格式很友善（Human-readable），但不適合給機器讀取。

    * 你會發現它連 `pip` 和 `setuptools` 這些「廚房工具」本身都列出來了，這通常不是你專案「真正」需要的。

`pip show requests` (單一食材的「產銷履歷」)

* 官方說明： 顯示特定套件的資訊。

* 實務情境 (「總管，幫我查一下『`requests`』這包食材的詳細資料」)：

    * 這是一個非常重要的除錯 (Debug) 工具。

    * `Version`:：讓你知道你現在用的到底是哪個版本。

    * `Location`:：讓你知道這個套件到底安裝在哪個路徑（確認你是否在正確的 `.venv` 裡）。

    * `Requires`:：最關鍵的資訊！ 它告訴你，`requests` 這個套件「它自己也依賴了哪些其他套件」。

    * 實務應用：

        * 你身為軟體工程師，一定會遇到「**依賴衝突 (Dependency Conflict)**」。

        * 例如：你裝的 `package-A` 需要 `requests==2.6.0`，但你新裝的 `package-B` 卻需要 `requests==2.7.0`。

        * 這時 `pip` 可能會報錯。你就必須用 `pip show package-A` 和 `pip show package-B` 去看它們各自的 `Requires`: 欄位，找出是哪個「孫子輩」的依賴發生了衝突，然後手動解決它。

`pip freeze` (給「機器」讀的精確清單)

* 官方說明： 產生一個 `pip install` 可讀懂的套件清單。

* 實務情境 (「總管，請產生一份標準化的『採購訂單』，精確到品牌和批號」)：

    * `pip freeze` 的輸出格式是 `package==version`。

    * 這就是為了**「100% 可重現性」**而生的。

    * 它只輸出你安裝的套件，而不會包含 `pip` 這類「內建工具」。

    * 你永遠不該手動去編輯 `pip freeze` 產生的內容。

---
4. `requirements.txt`: 你的「標準採購訂單」

這就是 Python 專案協作的基石。

`pip freeze > requirements.txt` (歸檔：將當前庫存存為標準訂單)

* 實務情境：

    * 你剛才安裝了 `requests==2.7.0`，並且測試都通過了。

    * 你立刻執行 `pip freeze > requirements.txt`。

    * 這會「覆蓋」掉舊的 `requirements.txt` 檔案，用你當前環境的「精確庫存清單」來更新它。

    * 然後你必須 `git commit requirements.txt`。這個檔案和你的原始碼一樣重要。

`pip install -r requirements.txt` (執行：照著訂單去採購)

* 實務情境 (這可能是這章節最重要的指令)：

    1. 新同事報到： 他 `git clone` 你的專案，接著建立虛擬環境 (`python -m venv .venv`)，啟動 (`source .venv/bin/activate`)，然後他唯一要做的就是執行 `pip install -r requirements.txt`。他就能在 5 分鐘內建置一個和你一模一樣的開發環境。

    2. CI/CD 部署： (這對你 R&D 職位很關鍵) 當你把程式碼推送到 GitLab/GitHub，自動化流程 (CI/CD Pipeline) 會啟動。它做的第一件事就是 `pip install -r requirements.txt`，確保在測試和打包 (Build) 應用時，使用的是你指定的「那套」依賴。

    3. 生產環境部署： 你的應用程式要上線到伺服器時，部署腳本執行的也是這個指令。

這確保了「在我電腦上可以跑 (It works on my machine)」這句工程師惡夢不會發生。

---
**專業實務進階：一個 `requirements.txt` 夠嗎？**

* 在實務上，尤其是在你接觸的 AI/ML 或大型系統中，只用一個 `requirements.txt` 是不夠的。

* 問題： `pip freeze` 會把「所有」套件都凍結起來。但有些套件是「開發時」才需要（例如 `pytest` 用於測試、`black` 用於程式碼排版、`jupyter` 用於數據分析），而「生產環境」執行時根本不需要它們。

* 業界標準做法： 使用多個 `requirements` 檔案。

    1. `requirements.in` (或 `setup.py`)

        * 你手動維護這個檔案，只寫「頂層」的依賴。

        * 例如：
            
            ```
            # requirements.in
            django
            requests
            ```

    2. `requirements.txt` (生產環境用)

        * 你使用工具 (例如 `pip-compile`，來自 `pip-tools` 套件) 來自動生成這個檔案。

        * `pip-compile requirements.in`

        * 它會自動幫你解析所有「孫子輩」的依賴，並產生一個像 `pip freeze` 一樣鎖定版本的完整清單。

    3. `requirements-dev.txt` (開發環境用)

        * 你手動建立這個檔案，用來管理「開發工具」。

        * 它會長這樣：
            ```
            # requirements-dev.txt

            # 引用並安裝所有生產環境的套件
            -r requirements.txt

            # 額外的開發工具
            pytest
            black
            mypy
            ```

* 你的工作流就變成：

    * 新同事 / CI / 生產環境： `pip install -r requirements.txt`

    * 你 (開發者)： `pip install -r requirements-dev.txt` (這樣你就同時裝了生產和開發的全部套件)

這就是 `pip` 在專業軟體開發中，從「會用」到「精通」的實務應用。